In [ ]:
import numpy as np
from scipy import interpolate
import scipy.io as sio
import matplotlib.pyplot as plt

In [ ]:
def creat_time_vec(data):
    t = np.arange(0, np.shape(data)[1], 1)
    # t = np.reshape(t, np.shape(data))
    data_new = interpolate.interp1d(t, data, kind="linear")
    return data_new.y,t

In [ ]:
energy = sio.loadmat(r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\Final Project\Project2-Omri and Idan\Results\06_30_2022_10_12_02_mu_0__0_2__2_8_Js__4_0_runs_2\2\3\energy_vec_mu_1.6_energy_4.0_run_num_1_total_num_target_2.mat")
energy = energy["foo"]

In [ ]:
data,time = creat_time_vec(energy)